In [1]:
import cv2
import numpy as np
!pip install opencv-python

In [6]:
cap = cv2.VideoCapture('station4.avi')

if not cap.isOpened():
    print("Video open failed!")
else :    
    #이미지 프레임 사이즈값을 받습니다.
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    #전체 프레임 수값을 받습니다.
    frame_cnt = round(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #초당 프레임 값을받습니다.
    fps = cap.get(cv2.CAP_PROP_FPS)
    #한 프레임당 걸리는 시간을 받습니다.
    delay = int(1000 / fps)
    #영상을 저장할 코덱정보를 저장합니다.
    fourcc = cv2.VideoWriter_fourcc(*'DIVX') # *'DIVX' == 'D', 'I', 'V', 'X'
    #영상을 저장할 이름을 지정합니다.
    outputVideo = cv2.VideoWriter('stationOutput.avi', fourcc, fps, (w, h))
    
    
    #전체 프레임까지 영상 루프를 돌립니다.
    for i in range(frame_cnt): 
        ret, frame = cap.read()
        
        
        if not ret:  break
        label='%d'%(i)
        
        frame = cv2.inRange(frame, (0, 150, 150), (150, 255, 255))#Yellow RGB 로 설정
        
        lines = cv2.HoughLinesP(frame, 1, np.pi / 180., 160, minLineLength=300, maxLineGap=5)
        #1번째 인자 : edges
        #2번째 인자 : rho로 1픽셀간격으로 한다. 보통 기본값
        #3번째 인자 : 각도는 1도단위. 보통 기본값
        #4번째 인자 : threshold는 160정도
        #5번째 인자 : 최소길이 50
        #6번째 인자 : 5픽셀정도는 떨어져도 이어라
        
        frame=cv2.cvtColor(frame,cv2.COLOR_GRAY2BGR)#다시 컬러영상으로 변환. 저장하기 위해 실제로는 할필요없음
        frame = cv2.GaussianBlur(frame, (0, 0), 1) #노이즈 완화
        
        if lines is not None:#혹시 라인이 하나도 검출이 안될수 있다.
            for j in range(lines.shape[0]):#shape의 첫번째가 직선성분의 개수
                pt1 = (lines[j][0][0], lines[j][0][1])  # 시작점 좌표 x,y
                pt2 = (lines[j][0][2], lines[j][0][3])  # 끝점 좌표 x,y
                #가운데 인자는 무조건 0이다. 더미개념
                cv2.line(frame, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

        #좌표는 (X,Y 임을 잊지맙시다)
        frame=cv2.putText(frame,label , (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255))
        cv2.imshow('', frame)
            
        outputVideo.write(frame)
        #프레임당 기존의 딜레이를 줍니다.
        cv2.waitKey(delay)
    
        

        if cv2.waitKey(10) == 27:  break   # ESC

outputVideo.release()
cap.release()
cv2.destroyAllWindows()